# Tutorial 06: Networks from OpenStreetMap

In this tutorial, we discuss how networks that have been imported from OpenStreetMap can be integrated and run in Flow. This will all be presented via the Bay Bridge network, seen in the figure below. Networks from OpenStreetMap are commonly used in many traffic simulators for the purposes of replicating traffic in realistic traffic geometries. This is true in both SUMO and Aimsun (which are both supported in Flow), with each supporting several techniques for importing such network files. This process is further simplified and abstracted in Flow, with users simply required to specify the path to the osm file in order to simulate traffic in the network.

在本教程中，我们将讨论如何集成和运行从OpenStreetMap导入的网络。这些都将通过海湾大桥网络呈现，如下图所示。来自OpenStreetMap的网络通常用于许多交通模拟器，目的是在真实的交通几何图形中复制交通。这在SUMO和Aimsun中都是正确的(在Flow中都支持)，每个都支持导入这样的网络文件的一些技术。这个过程在Flow中得到了进一步的简化和抽象，用户只需指定osm文件的路径，就可以模拟网络中的流量。


<img src="img/bay_bridge_osm.png" width=750>
<center> **Figure 1**: Snapshot of the Bay Bridge from OpenStreetMap </center>

Before we begin, let us import all relevant Flow parameters as we have done for previous tutorials. If you are unfamiliar with these parameters, you are encouraged to review tutorial 1.
在开始之前，让我们像在之前的教程中那样导入所有相关的流参数。如果您不熟悉这些参数，建议您阅读教程1。

In [ ]:
# the TestEnv environment is used to simply simulate the network
from flow.envs import TestEnv

# the Experiment class is used for running simulations
from flow.core.experiment import Experiment

# all other imports are standard
from flow.core.params import VehicleParams
from flow.core.params import NetParams
from flow.core.params import InitialConfig
from flow.core.params import EnvParams
from flow.core.params import SumoParams

## 1. Running a Default Simulation 运行默认模拟

In order to create a network object in Flow with network features depicted from OpenStreetMap, we will use the base `Network` class. This class can sufficiently support the generation of any .osm file.

为了在Flow中创建具有OpenStreetMap描述的网络特性的网络对象，我们将使用基本的“network”类。此类可以充分支持任何.osm文件的生成。

In [ ]:
from flow.networks import Network

In order to recreate the network features of a specific osm file, the path to the osm file must be specified in `NetParams`. For this example, we will use an osm file extracted from the section of the Bay Bridge as depicted in Figure 1.

In order to specify the path to the osm file, simply fill in the `osm_path` attribute with the path to the .osm file as follows:
为了重新创建特定osm文件的网络特性，必须在“NetParams”中指定osm文件的路径。对于本例，我们将使用从Bay Bridge中提取的osm文件，如图1所示。


为了指定osm文件的路径，只需用.osm文件的路径填充' osm_path '属性，如下所示:

In [ ]:
net_params = NetParams(
    osm_path='networks/bay_bridge.osm'
)

Next, we create all other parameters as we have in tutorials 1 and 2. For this example, we will assume a total of 1000 are uniformly spread across the Bay Bridge. Once again, if the choice of parameters is unclear, you are encouraged to review Tutorial 1.

接下来，我们创建教程1和2中的所有其他参数。对于本例，我们假设海湾大桥上平均分布了1000个节点。同样，如果参数的选择不明确，建议您阅读教程1。

In [ ]:
# create the remainding parameters
env_params = EnvParams()
sim_params = SumoParams(render=True)
initial_config = InitialConfig()
vehicles = VehicleParams()
vehicles.add('human', num_vehicles=100)

We are finally ready to test our network in simulation. In order to do so, we create an `Experiment` object and run the simulation for a number of steps. This is done in the cell below.

我们终于可以在模拟中测试我们的网络了。为了做到这一点，我们创建了一个“实验”对象，并对多个步骤运行模拟。这是在下面的单元格中完成的。

In [ ]:
flow_params = dict(
    exp_tag='bay_bridge',
    env_name=TestEnv,
    network=Network,
    simulator='traci',
    sim=sim_params,
    env=env_params,
    net=net_params,
    veh=vehicles,
    initial=initial_config,
)

# number of time steps
flow_params['env'].horizon = 1000
exp = Experiment(flow_params)

# run the sumo simulation
_ = exp.run(1)

## 2. Customizing the Network 自定义网络

While the above example does allow you to view the network within Flow, the simulation is limited for two reasons. For one, vehicles are placed on all edges within the network; if we wished to simulate traffic solely on the on the bridge and do not care about the artireols, for instance, this would result in unnecessary computational burdens. Next, as you may have noticed if you ran the above example to completion, routes in the base network class are defaulted to consist of the vehicles' current edges only, meaning that vehicles exit the network as soon as they reach the end of the edge they are originated on. In the next subsections, we discuss how the network can be modified to resolve these issues.

虽然上面的示例允许您查看流中的网络，但是由于两个原因，模拟是有限的。首先，车辆被放置在网络的所有边缘;例如，如果我们希望仅在桥上模拟交通，而不关心artireols，这将导致不必要的计算负担。接下来，正如您可能已经注意到的，如果您运行上面的例子来完成，那么基本网络类中的路由默认为仅由车辆的当前边组成，这意味着车辆一旦到达它们最初所在的边的末端，就会退出网络。在下一小节中，我们将讨论如何修改网络来解决这些问题。

### 2.1 Specifying Traversable Edges 指定可反驳的边缘

In order to limit the edges vehicles are placed on to the road sections edges corresponding to the westbound Bay Bridge, we define an `EDGES_DISTRIBUTION` variable. This variable specifies the names of the edges within the network that vehicles are permitted to originated in, and is assigned to the network via the `edges_distribution` component of the `InitialConfig` input parameter, as seen in the code snippet below. Note that the names of the edges can be identified from the .osm file or by right clicking on specific edges from the SUMO gui (see the figure below).

为了限制车辆被放置在与西行湾大桥相对应的路段边缘上，我们定义了一个‘EDGES_DISTRIBUTION’变量。此变量指定允许车辆起源于网络中的边缘的名称，并通过“InitialConfig”输入参数的“edges_distribution”组件分配给网络，如下面的代码片段所示。请注意，可以从.osm文件或通过右键单击SUMO gui中的特定边缘来标识边缘的名称(参见下图)。

<img src="img/osm_edge_name.png" width=600>
<center> **Figure 2**: Name of an edge from SUMO </center>

In [ ]:
# we define an EDGES_DISTRIBUTION variable with the edges within 
# the westbound Bay Bridge 
EDGES_DISTRIBUTION = [
    "11197898",
    "123741311", 
    "123741303",
    "90077193#0",
    "90077193#1", 
    "340686922", 
    "236348366", 
    "340686911#0",
    "340686911#1",
    "340686911#2",
    "340686911#3",
    "236348361", 
    "236348360#0", 
    "236348360#1"
]

# the above variable is added to initial_config
new_initial_config = InitialConfig(
    edges_distribution=EDGES_DISTRIBUTION
)

### 2.2 Creating Custom Routes 创建自定义的路线

Next, we choose to specify the routes of vehicles so that they can traverse the entire Bay Bridge, instead of the only the edge they are currently on. In order to this, we create a new network class that inherits all its properties from `Network` and simply redefine the routes by modifying the `specify_routes` variable. This method was originally introduced in Tutorial 07: Creating Custom Network. The new network class looks as follows:

接下来，我们选择指定车辆的路线，这样它们就可以穿越整个海湾大桥，而不是仅仅是它们目前所在的边缘。为此，我们创建了一个新的网络类，它继承了“network”的所有属性，并通过修改“specify_routes”变量重新定义路由。这个方法最初是在教程07中介绍的:创建自定义网络。新的网络类如下:

In [ ]:
# we create a new network class to specify the expected routes
class BayBridgeOSMNetwork(Network):

    def specify_routes(self, net_params):
        return {
            "11197898": [
                "11197898", "123741311", "123741303", "90077193#0", "90077193#1", 
                "340686922", "236348366", "340686911#0", "340686911#1",
                "340686911#2", "340686911#3", "236348361", "236348360#0", "236348360#1",
            ],
            "123741311": [
                "123741311", "123741303", "90077193#0", "90077193#1", "340686922", 
                "236348366", "340686911#0", "340686911#1", "340686911#2",
                "340686911#3", "236348361", "236348360#0", "236348360#1"
            ],
            "123741303": [
                "123741303", "90077193#0", "90077193#1", "340686922", "236348366",
                "340686911#0", "340686911#1", "340686911#2", "340686911#3", "236348361",
                "236348360#0", "236348360#1"
            ],
            "90077193#0": [
                "90077193#0", "90077193#1", "340686922", "236348366", "340686911#0",
                "340686911#1", "340686911#2", "340686911#3", "236348361", "236348360#0",
                "236348360#1"
            ],
            "90077193#1": [
                "90077193#1", "340686922", "236348366", "340686911#0", "340686911#1",
                "340686911#2", "340686911#3", "236348361", "236348360#0", "236348360#1"
            ],
            "340686922": [
                "340686922", "236348366", "340686911#0", "340686911#1", "340686911#2",
                "340686911#3", "236348361", "236348360#0", "236348360#1"
            ],
            "236348366": [
                "236348366", "340686911#0", "340686911#1", "340686911#2", "340686911#3",
                "236348361", "236348360#0", "236348360#1"
            ],
            "340686911#0": [
                "340686911#0", "340686911#1", "340686911#2", "340686911#3", "236348361",
                "236348360#0", "236348360#1"
            ],
            "340686911#1": [
                "340686911#1", "340686911#2", "340686911#3", "236348361", "236348360#0",
                "236348360#1"
            ],
            "340686911#2": [
                "340686911#2", "340686911#3", "236348361", "236348360#0", "236348360#1"
            ],
            "340686911#3": [
                "340686911#3", "236348361", "236348360#0", "236348360#1"
            ],
            "236348361": [
                "236348361", "236348360#0", "236348360#1"
            ],
            "236348360#0": [
                "236348360#0", "236348360#1"
            ],
            "236348360#1": [
                "236348360#1"
            ]
        }

### 2.3 Rerunning the Simulation 运行仿真

We are now ready to rerun the simulation with fully defined vehicle routes and a limited number of traversable edges. If we run the cell below, we can see the new simulation in action.

现在，我们准备使用完全定义的车辆路线和有限数量的可遍历边重新运行模拟。如果我们运行下面的单元格，我们可以看到新的模拟正在运行。

In [ ]:
flow_params = dict(
    exp_tag='bay_bridge',
    env_name=TestEnv,
    network=BayBridgeOSMNetwork,
    simulator='traci',
    sim=sim_params,
    env=env_params,
    net=net_params,
    veh=vehicles,
    initial=new_initial_config,
)

# number of time steps
flow_params['env'].horizon = 10000
exp = Experiment(flow_params)

# run the sumo simulation
_ = exp.run(1)

## 3. Other Tips 其他技巧

This tutorial introduces how to incorporate OpenStreetMap files in Flow. This feature, however, does not negate other features that are introduced in other tutorials and documentation. For example, if you would like to not have vehicles be originated side-by-side within a network, this can still be done by specifying a "random" spacing for vehicles as follows:

本教程介绍如何将OpenStreetMap文件合并到Flow中。但是，这个特性并不否定其他教程和文档中介绍的其他特性。例如，如果您不想让车辆在网络中并排产生，仍然可以为车辆指定一个“随机”间距，如下所示:

    initial_config = InitialConfig(
        spacing="random",
        edges_distribution=EDGES_DISTRIBUTION
    )

In addition, inflows of vehicles can be added to networks imported from OpenStreetMap as they are for any other network (see the tutorial on adding inflows for more on this).
此外，车辆的流入可以添加到从OpenStreetMap导入的网络中，就像添加其他网络一样(更多信息请参阅添加流入的教程)。